In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random

In [2]:
pages = set()

In [3]:
random.seed(datetime.datetime.now())

In [4]:
# 获取页面所有的内链的列表
def getInternalLinks(bsObj, includeUrl):
    internaliLinks = []
    # 找出所有以 "/" 开头的链接
    for link in bsObj.findAll('a', href=re.compile("^(/|.*)" + includeUrl + ")")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internaliLinks:
                internaliLinks.append(link.attrs['href'])
    return internaliLinks

In [5]:
# 获取页面所有外链的列表
def getExternalLinks(bsObj, excludeUrl):
    externalLinks = []
    # 找出所有以 "http" 或 "www" 开头且不包含当前URL的链接
    for link in bsObj.findAll("a", href=re.compile("^(http|www)((?!"+excludeUrl+").)*$")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

In [6]:
def splitAddress(address):
    addressParts = address.replace('http://', '').split('/')
    return addressParts

In [7]:
def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bsObj = BeautifulSoup(html)
    externalLinks = getExternalLinks(bsObj, splitAddress(startingPage)[0])
    if len(externalLinks) == 0:
        internalLinks = getInternalLinks(startingPage)
        return getNextExternalLink(internalLinks[random.randint(0, len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]

In [8]:
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink("http://oreilly.com")
    print('随机外链是:' + externalLink)
    followExternalOnly(externalLink)

In [9]:
followExternalOnly("http://oreilly.com")

随机外链是:https://www.youtube.com/user/OreillyMedia
随机外链是:https://www.youtube.com/user/OreillyMedia
随机外链是:https://www.amazon.com/OReilly-Media-Inc/dp/B087YYHL5C/ref=sr_1_2?dchild=1&keywords=oreilly&qid=1604964116&s=mobile-apps&sr=1-2
随机外链是:https://www.youtube.com/user/OreillyMedia
随机外链是:https://itunes.apple.com/us/app/safari-to-go/id881697395
随机外链是:https://www.youtube.com/user/OreillyMedia
随机外链是:https://www.amazon.com/OReilly-Media-Inc/dp/B087YYHL5C/ref=sr_1_2?dchild=1&keywords=oreilly&qid=1604964116&s=mobile-apps&sr=1-2
随机外链是:https://play.google.com/store/apps/details?id=com.safariflow.queue
随机外链是:https://www.youtube.com/user/OreillyMedia
随机外链是:https://channelstore.roku.com/details/c8a2d0096693eb9455f6ac165003ee06/oreilly
随机外链是:https://www.youtube.com/user/OreillyMedia
随机外链是:https://channelstore.roku.com/details/c8a2d0096693eb9455f6ac165003ee06/oreilly
随机外链是:https://channelstore.roku.com/details/c8a2d0096693eb9455f6ac165003ee06/oreilly
随机外链是:https://channelstore.roku.com/details/c8a2d0096

KeyboardInterrupt: 